In [13]:
import sys
import json
import math
import argparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import random

In [150]:
file_ftm_nobody = "outputftm_27"
# file_ftm_nobody = "log.ftm.nobody.csv",

# file_ftm_1p = "log.ftm.1person.csv",
# file_ftm_2p = "log.ftm.2person.csv"

In [151]:
def read_file(filename):
    lines = []
    with open(filename) as f:
        while True:
            line = f.readline()
            components = line.split(",")
            if line != '':
                lines.append(components)
            if not line:
                break;
    return lines

In [152]:
def get_data_pos(char_string, char_compare):
    for i in range(len(char_string)):
        if char_string[i] == char_compare:
            return i
    print(char_string)

In [153]:
    lines = []
    with open("outputftm_27") as f:
        while True:
            line = f.readline()
            components = line.split(",")
            if line != '':
                lines.append(components)
            if not line:
                break;

In [154]:
lines_old = lines

In [166]:
    components = lines[3]
    channel = components[1]
    mac = components[2]
    rtt_time = components[3]
    
#     j = get_data_pos(components[5], '[')
    print(type(j))
#     print(components[5][j+1:-1])
    rtt_dist = components[4]
        # collect rtt data:\n",
    rtt_data = []
    rtt_data.append(components[5][5:-1])
    
    for i in range(6, len(components)-1):
        rtt_data.append(components[i])
    
#     m = get_data_pos(components[-1], ']')
#     rtt_data.append(components[-1][1:m])

<class 'int'>


In [168]:
def process_ftm(components):
    #     print(components)\n",
    channel = components[1]
    mac = components[2]
    rtt_time = components[3]
    
#     j = get_data_pos(components[5], '[') +1
    
    rtt_dist = components[4]
        # collect rtt data:\n",
    rtt_data = []
    rtt_data.append(components[5][5:-1])
    
    for i in range(6, len(components)-1):
        rtt_data.append(components[i])
        
    return channel, mac, rtt_time, rtt_dist, rtt_data

In [186]:
lines[1:][1]

['FTM_DATA',
 '2',
 '7c:df:a1:06:99:97',
 '45',
 '6.76',
 '" ""[57813"',
 '59375',
 '59376',
 '59376',
 '59375',
 '59376',
 '59376',
 '59375',
 '60938',
 '59375',
 '59376',
 '60938',
 '62500',
 '60938',
 '60938',
 '60938',
 '60938',
 '60938',
 '62501',
 '62501',
 '62501',
 '64063',
 '67188',
 '67188',
 '67188',
 '67188',
 '65626',
 '67188',
 '67188',
 '"1073544996]"""\n']

In [205]:
    parsed_data = []
    divide_index = 10
    truth = 0
    for i in range(len(lines[1:])):
#         print(i)
        channel, mac, rtt_time, rtt_dist, rtt_data = process_ftm(lines[1:][i])
        
        parsed_data.append([channel, mac, rtt_time, rtt_dist, rtt_data])
#     print(parsed_data)
    df_ftm = pd.DataFrame(parsed_data, columns=['channel', 'mac', 'rtt_time', 'rtt_dist', 'rtt_data'])

    ftm_w_truth = []
    ftm_w_o_truth = []
    for i in range(len(lines)//divide_index):
        temp_lst = []
        for j in range(divide_index):
            temp_lst.append(float(df_ftm.iloc[10*i+j]['rtt_dist']))
        ftm_w_o_truth.append(temp_lst)

In [206]:
df_ftm

,channel,mac,rtt_time,rtt_dist,rtt_data
0,1,7c:df:a1:06:99:97,208,31.30,"[270313, 268750, 268750, 268751, 270313, 27031..."
1,2,7c:df:a1:06:99:97,45,6.76,"[57813, 59375, 59376, 59376, 59375, 59376, 593..."
2,3,7c:df:a1:06:99:97,38,5.72,"[57813, 56250, 56251, 56251, 46875, 56251, 546..."
3,4,7c:df:a1:06:99:97,43,6.50,"[60938, 62500, 62501, 62501, 60938, 60938, 609..."
4,5,7c:df:a1:06:99:97,18,2.73,"[32813, 32813, 32813, 32813, 31250, 32813, 328..."
...,...,...,...,...,...
715,6,7c:df:a1:06:99:97,23,3.51,"[37500, 39063, 37500, 37501, 37501, 39063, 359..."
716,7,7c:df:a1:06:99:97,30,4.55,"[45313, 45313, 46876, 45313, 45313, 45313, 453..."
717,8,7c:df:a1:06:99:97,26,3.90,"[40625, 40626, 42188, 40625, 40625, 40626, 406..."
718,9,7c:df:a1:06:99:97,32,4.81,"[46875, 45313, 48438, 46875, 46875, 48438, 468..."


In [196]:
def generate_dataframe_w_truth(lines, divide_index, truth):
    parsed_data = []
    for i in range(len(lines[1:])):
        channel, mac, rtt_time, rtt_dist, rtt_data = process_ftm(lines[1:][i])
        parsed_data.append([channel, mac, rtt_time, rtt_dist, rtt_data])

    df_ftm = pd.DataFrame(parsed_data, columns=['channel', 'mac', 'rtt_time', 'rtt_dist', 'rtt_data'])

    ftm_w_truth = []
    ftm_w_o_truth = []
    for i in range(len(lines)//divide_index):
        temp_lst = []
        for j in range(divide_index):
            temp_lst.append(float(df_ftm.iloc[10*i+j]['rtt_dist']))
        ftm_w_o_truth.append(temp_lst)
#         ftm_w_truth.append([temp_lst, truth])\n",
#     ftm_w_truth = pd.DataFrame(ftm_w_truth, columns = ['rtt_dist', 'ground truth'])\n",
#     print(ftm_w_truth)\n",
    return ftm_w_o_truth

In [204]:
 #read file and generate
lines_nobody = read_file(file_ftm_nobody)
ftm_nobody_w_truth = generate_dataframe_w_truth(lines_nobody, 10, 0)

In [207]:
ftm_nobody_w_truth

[[31.3, 6.76, 5.72, 6.5, 2.73, 2.6, 4.29, 3.12, 4.29, 5.46],
 [5.2, 6.11, 5.59, 5.46, 3.51, 2.73, 4.55, 4.29, 5.46, 5.2],
 [8.06, 11.18, 6.76, 9.88, 4.68, 3.25, 5.33, 4.29, 4.81, 5.59],
 [4.68, 6.5, 3.38, 4.16, 4.03, 3.64, 5.33, 3.77, 4.55, 6.11],
 [6.76, 17.9, 6.76, 4.55, 5.59, 6.24, 5.46, 3.9, 4.94, 5.72],
 [14.04, 13.0, 7.54, 6.89, 4.81, 3.25, 5.07, 4.42, 4.94, 5.33],
 [5.07, 5.46, 3.12, 5.2, 5.07, 4.03, 4.81, 4.29, 4.68, 5.33],
 [6.11, 7.28, 5.46, 6.11, 4.68, 4.03, 4.81, 3.9, 4.16, 5.07],
 [4.94, 2.6, 1.95, 1.95, 2.86, 2.99, 4.29, 3.64, 4.29, 4.94],
 [4.55, 2.99, 2.21, 1.56, 2.86, 3.12, 4.55, 4.16, 5.07, 5.46],
 [5.07, 10.79, 6.37, 8.32, 5.46, 3.51, 4.03, 4.42, 5.07, 5.59],
 [8.84, 6.5, 9.75, 7.28, 5.46, 4.03, 4.55, 4.42, 5.07, 5.59],
 [9.36, 6.63, 9.88, 7.15, 5.59, 4.16, 4.55, 4.42, 5.07, 5.33],
 [9.1, 6.5, 9.75, 7.02, 5.33, 4.03, 4.55, 4.55, 5.07, 5.59],
 [9.23, 6.37, 9.75, 6.89, 5.59, 4.03, 4.55, 4.42, 5.07, 5.59],
 [9.49, 6.5, 9.88, 7.41, 5.72, 4.16, 4.55, 4.42, 5.07, 5.59],
 [

In [199]:
# # combine and shuffle all data
# all_ftm_data = []
# for i in range(len(ftm_nobody_w_truth)):
#     all_ftm_data.append(ftm_nobody_w_truth[i])


In [225]:
with open('ftm_processed_data.csv', 'w') as filehandle:
     for item in ftm_nobody_w_truth:
        item_str = str(item)
        filehandle.writelines(item_str[1:-1]+ '\n')
#         filehandle.writelines(\"%s\\n\" % item_str[1:-1])
#     filehandle.writelines(\"%s\\n\" % place for place in all_ftm_data)\n",
filehandle.close()

[31.3, 6.76, 5.72, 6.5, 2.73, 2.6, 4.29, 3.12, 4.29, 5.46]
[5.2, 6.11, 5.59, 5.46, 3.51, 2.73, 4.55, 4.29, 5.46, 5.2]
[8.06, 11.18, 6.76, 9.88, 4.68, 3.25, 5.33, 4.29, 4.81, 5.59]
[4.68, 6.5, 3.38, 4.16, 4.03, 3.64, 5.33, 3.77, 4.55, 6.11]
[6.76, 17.9, 6.76, 4.55, 5.59, 6.24, 5.46, 3.9, 4.94, 5.72]
[14.04, 13.0, 7.54, 6.89, 4.81, 3.25, 5.07, 4.42, 4.94, 5.33]
[5.07, 5.46, 3.12, 5.2, 5.07, 4.03, 4.81, 4.29, 4.68, 5.33]
[6.11, 7.28, 5.46, 6.11, 4.68, 4.03, 4.81, 3.9, 4.16, 5.07]
[4.94, 2.6, 1.95, 1.95, 2.86, 2.99, 4.29, 3.64, 4.29, 4.94]
[4.55, 2.99, 2.21, 1.56, 2.86, 3.12, 4.55, 4.16, 5.07, 5.46]
[5.07, 10.79, 6.37, 8.32, 5.46, 3.51, 4.03, 4.42, 5.07, 5.59]
[8.84, 6.5, 9.75, 7.28, 5.46, 4.03, 4.55, 4.42, 5.07, 5.59]
[9.36, 6.63, 9.88, 7.15, 5.59, 4.16, 4.55, 4.42, 5.07, 5.33]
[9.1, 6.5, 9.75, 7.02, 5.33, 4.03, 4.55, 4.55, 5.07, 5.59]
[9.23, 6.37, 9.75, 6.89, 5.59, 4.03, 4.55, 4.42, 5.07, 5.59]
[9.49, 6.5, 9.88, 7.41, 5.72, 4.16, 4.55, 4.42, 5.07, 5.59]
[9.49, 6.5, 9.75, 7.15, 5.59, 4.03